## **Role:** The Master Report. This notebook loads all the JSON metrics we created (`results_detector.json`, `human_annotations.jsonl`, `results_impact.json`) and prints the final Scorecard.

## 1: Setup

In [1]:
import sys
import os
import json
import pandas as pd
from IPython.display import display, Markdown

print("✅ Report Generator Loaded")

✅ Report Generator Loaded


## 2: Run Missing Steps (Automated)

In [2]:
# Ensure Impact is calculated
!python impact_simulator.py

💰 Calculating Business Impact (Recovery Rate: 30.0%)
{
  "metric": "Business Impact",
  "anomalies_processed": 5,
  "negative_events": 0,
  "total_revenue_at_risk": 0.0,
  "estimated_annual_recovery": 0.0,
  "one_time_recovery": 0.0,
  "actions_automated": 5
}


## 3: Load All Metrics

In [3]:
def load_json(path):
    try:
        with open(path, 'r') as f: return json.load(f)
    except: return {}

# 1. Automated Technical Metrics
detector = load_json("results_detector.json")
schema = load_json("results_schema.json")
impact = load_json("results_impact.json")

# 2. Human Ratings (JSONL)
human_scores = []
try:
    with open("human_annotations.jsonl", "r") as f:
        for line in f: human_scores.append(json.loads(line))
    avg_human = pd.DataFrame(human_scores)['human_score'].mean() if human_scores else 0
except:
    avg_human = 0

print("✅ Metrics Aggregated")

✅ Metrics Aggregated


## 4: The Final Scorecard

In [4]:
# Display the "Winning Submission" Table
md = f"""
# 🏆 Capstone Evaluation Report: SalesOps Agent

## 1. Technical Quality (Implementation)
| Metric | Value | Goal | Status |
| :--- | :--- | :--- | :--- |
| **Detector F1 Score** | {detector.get('f1_score', 0):.2f} | > 0.75 | {'✅' if detector.get('f1_score', 0) > 0.75 else '⚠️'} |
| **Explainer Compliance** | {schema.get('compliance_rate', 0)*100:.0f}% | 100% | {'✅' if schema.get('compliance_rate') == 1 else '⚠️'} |
| **Human Helpfulness** | {avg_human:.1f}/5.0 | > 3.5 | {'✅' if avg_human > 3.5 else '⚠️'} |

## 2. Business Impact (Value)
* **Anomalies Processed:** {impact.get('anomalies_processed', 0)}
* **Revenue at Risk (Sample):** ${impact.get('total_revenue_at_risk', 0):,.2f}
* **Estimated Recovery (Annualized):** **${impact.get('estimated_annual_recovery', 0):,.2f}** 💰

## 3. Reliability & Safety
* **PII Redaction:** Active (Verified in Day 8)
* **Idempotency:** Active (Verified in Day 6)
* **Observability:** Full Traces Available (Day 9)

---
*Report generated via `evaluation/99_evaluation_report.ipynb`*
"""
display(Markdown(md))


# 🏆 Capstone Evaluation Report: SalesOps Agent

## 1. Technical Quality (Implementation)
| Metric | Value | Goal | Status |
| :--- | :--- | :--- | :--- |
| **Detector F1 Score** | 0.00 | > 0.75 | ⚠️ |
| **Explainer Compliance** | 100% | 100% | ✅ |
| **Human Helpfulness** | 0.0/5.0 | > 3.5 | ⚠️ |

## 2. Business Impact (Value)
* **Anomalies Processed:** 5
* **Revenue at Risk (Sample):** $0.00
* **Estimated Recovery (Annualized):** **$0.00** 💰

## 3. Reliability & Safety
* **PII Redaction:** Active (Verified in Day 8)
* **Idempotency:** Active (Verified in Day 6)
* **Observability:** Full Traces Available (Day 9)

---
*Report generated via `evaluation/99_evaluation_report.ipynb`*


## 5: Export Summary for Writeup

In [5]:
# Save the final summary for the README
final_summary = {
    "technical": detector,
    "quality": {"human_score": avg_human, "schema": schema},
    "business": impact
}
with open("evaluation_summary.json", "w") as f:
    json.dump(final_summary, f, indent=2)
    
print("✅ evaluation_summary.json saved")

✅ evaluation_summary.json saved
